In [83]:
from sklearn import preprocessing
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import poisson
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, classification_report
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
xgb.set_config(verbosity=0)
from sklearn.feature_selection import RFECV
import random
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score
from sklearn import metrics
import matplotlib
from sklearn.model_selection import train_test_split 
pd.set_option('display.max_columns', None)

In [84]:
# First part is merging all meetings
datasets = """Bed002 Bed003 Bed004 Bed005 Bed006 Bed008 Bed009 Bed010 Bed011 Bed012 Bed013 Bed014 Bed015 Bed016 Bed017 Bmr001 Bmr002 Bmr005 Bmr007 Bmr009 Bmr010 Bmr011 Bmr012 Bmr013 Bmr014 Bmr018 Bmr019 Bmr021 Bmr022 Bmr024 Bmr025 Bmr026 Bmr027 Bmr029 Bns001 Bns002""".split(" ")
results_merged_path = "../social-signal-processing-project/results_merged/"

all_df = pd.DataFrame()
for elem in datasets:
    path = (os.path.realpath(os.path.join(os.getcwd(), (f"{results_merged_path}"+ elem + ".csv"))))
    df = pd.read_csv(path, sep=';')
    all_df = pd.concat([all_df,df], ignore_index=True)

In [85]:
#change boolean to 1 or 0
all_df['speakerChange'] = all_df["speakerChange"].astype(float)
all_df['boundary'] = all_df["boundary"].astype(float)

#keep only useful columns (can't use f0_means, f0_stds, and similarity yet as they are lists)
#normal split (but WARNING!!! NBR OF BOUNDARY == 1 IS VERYYYY LOW)
# X_df = all_df[['StartTimeA','EndTimeA','StartTimeB','EndTimeB','pause','speakerChange']]
# y_df = all_df['boundary']

#test using the same amount of rows with boundary==1 and ==0
nbr_1 = len(all_df.loc[all_df['boundary']==1])
df_0 = all_df.loc[all_df['boundary']==0].head(nbr_1)
df_1 = all_df.loc[all_df['boundary']==1]
df_balanced = pd.concat([df_0,df_1], ignore_index=True)
df_balanced.reset_index(drop=True,inplace=True)

X_df = df_balanced[['StartTimeA','EndTimeA','StartTimeB','EndTimeB','pause','speakerChange']]
y_df = df_balanced['boundary']

#check dataset statistics
print(y_df.describe())
print(X_df.corrwith(y_df))
print(X_df.loc[0])
print(X_df.describe())

#split dataset
X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(X_df, y_df, test_size = 0.20, random_state = 97,stratify=y_df)
X_train_40, X_test_40, y_train_40, y_test_40 = train_test_split(X_df, y_df, test_size = 0.40, random_state = 97,stratify=y_df)

count    916.000000
mean       0.500000
std        0.500273
min        0.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        1.000000
Name: boundary, dtype: float64
StartTimeA       0.549669
EndTimeA         0.550042
StartTimeB       0.549596
EndTimeB         0.549842
pause            0.065577
speakerChange    0.376063
dtype: float64
StartTimeA       5.790
EndTimeA         6.200
StartTimeB       6.393
EndTimeB         7.043
pause            0.193
speakerChange    1.000
Name: 0, dtype: float64
        StartTimeA     EndTimeA   StartTimeB     EndTimeB       pause  \
count   916.000000   915.000000   916.000000   916.000000  915.000000   
mean   1082.017647  1084.349883  1084.598391  1087.256819    0.611268   
std     951.011860   951.741243   951.225975   951.402978    3.475570   
min       5.790000     6.200000     6.393000     7.043000  -85.570000   
25%     385.637250   386.212000   388.264750   389.549750    0.000000   
50%     819.589000   826.394000   825.

In [86]:
def XGB_class(X_train, X_test, y_train, y_test,learning_rate, n_estimators, max_depth,min_child_weight, gamma, subsample, colsample_bytree, simple):
    
    if simple:
        clf = xgb.XGBClassifier(seed = 24, use_label_encoder =False)
    else:
        clf = xgb.XGBClassifier(learning_rate = learning_rate, n_estimators = int(n_estimators), max_depth = int(max_depth), 
                                min_child_weight = min_child_weight, gamma = gamma, subsample = subsample, 
                                colsample_bytree = colsample_bytree, seed = 24,eval_metric='mlogloss',use_label_encoder =False)

    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)

    print(classification_report(y_test, y_predicted))
    print("Accuracy:", accuracy_score(y_test, y_predicted))
    print("MCC =",matthews_corrcoef(y_test, y_predicted))
    print("AUC =",metrics.roc_auc_score(y_test, y_predicted))
    
    return clf.predict_proba(X_test)[:,1]

In [87]:
XGBc_y_predicted_20 = XGB_class(X_train_20,X_test_20,y_train_20,y_test_20,0,0,0,0,0,0,0,simple=True)
XGBc_y_predicted_40 = XGB_class(X_train_40,X_test_40,y_train_40,y_test_40,0,0,0,0,0,0,0,simple=True)

C:\Users\marti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\marti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


              precision    recall  f1-score   support

         0.0       0.85      0.89      0.87        92
         1.0       0.89      0.84      0.86        92

    accuracy                           0.86       184
   macro avg       0.87      0.86      0.86       184
weighted avg       0.87      0.86      0.86       184

Accuracy: 0.8641304347826086
MCC = 0.7293387850622899
AUC = 0.8641304347826086
              precision    recall  f1-score   support

         0.0       0.84      0.88      0.86       184
         1.0       0.87      0.83      0.85       183

    accuracy                           0.85       367
   macro avg       0.85      0.85      0.85       367
weighted avg       0.85      0.85      0.85       367

Accuracy: 0.8528610354223434
MCC = 0.7067267096157344
AUC = 0.8527856973152768


In [88]:
print(XGBc_y_predicted_20)

[0.05509883 0.38894403 0.9987674  0.6440127  0.99681515 0.12353405
 0.01630569 0.01630569 0.9647402  0.99681515 0.09274486 0.06772611
 0.15069826 0.04608585 0.99826366 0.082968   0.6746042  0.9988558
 0.6096827  0.87642014 0.12205827 0.01664189 0.18968563 0.00845762
 0.30182478 0.989597   0.27458394 0.9987674  0.9988079  0.40125638
 0.19205125 0.9958319  0.10581758 0.06121317 0.9959317  0.02677582
 0.99668854 0.99894637 0.9814108  0.02357747 0.16976742 0.12406015
 0.4666691  0.9987674  0.9988244  0.6266333  0.9940625  0.9976659
 0.09737234 0.00845762 0.12415402 0.99799675 0.9982653  0.02676754
 0.02863378 0.6888694  0.28424534 0.21916924 0.99799675 0.01630569
 0.01630569 0.28835917 0.04327187 0.02714457 0.9985638  0.0164698
 0.00276963 0.26400766 0.9978752  0.99317545 0.91990054 0.10171605
 0.9945886  0.01365484 0.99799675 0.38817155 0.66258454 0.13520667
 0.4476232  0.1153617  0.9989084  0.03721153 0.44371867 0.99862945
 0.9969432  0.27310315 0.9959085  0.01004513 0.55192447 0.7859790